Construct Entry | EC number | Sequence csv for erxprot as for CLEAN's datasets

In [11]:
from src.utils import load_json
from collections import defaultdict
import pandas as pd
import subprocess
import os
import torch

In [45]:
# Create csv index in format of other datasets

f = load_json('../data/erxprot/sequences/sequences.json')
id2ec = defaultdict(list)
id2seq = {}
for ec, v in f.items():
    for elt in v:
        id = list(elt.keys())[0]
        seq = elt[id]
        id2ec[id].append(ec)
        id2seq[id] = seq

print(len(id2ec), len(id2seq))

data = []
for k in id2ec.keys():
    data.append([k, ';'.join(id2ec[k]), id2seq[k]])

header = ['Entry', 'EC number', 'Sequence']
df = pd.DataFrame(data, columns=header)
df.to_csv('../data/erxprot/erxprot.csv', index=False, sep='\t')

104200 104200


In [15]:
# # Re-name erxprot pt files ONLY RUN ONCE

# for dir in ['../data/erxprot/clean/', '../data/erxprot/esm/']:
#     for fn in os.listdir(dir):
#         id = fn.split('|')[0].strip()
#         cmd = ["mv", dir + fn, dir + id + '.pt']
#         subprocess.run(cmd)

In [7]:
# Overlap in enzymes

erx_csv = pd.read_csv('../data/erxprot/erxprot.csv', delimiter='\t')
erx_csv.set_index('Entry', inplace=True)

swissprot = pd.read_csv('../data/swissprot/swissprot.csv', delimiter='\t')
swissprot.set_index('Entry', inplace=True)

print(len(set(erx_csv.index) & set(swissprot.index)) / len(set(erx_csv.index) | set(swissprot.index)))
idx_intersect = list(set(erx_csv.index) & set(swissprot.index))

0.14423657628370382


In [3]:
# Overlap in EC numbers

erx_csv = pd.read_csv('../data/erxprot/erxprot.csv', delimiter='\t')
erx_csv.set_index('Entry', inplace=True)

swissprot = pd.read_csv('../data/swissprot/swissprot.csv', delimiter='\t')
swissprot.set_index('Entry', inplace=True)

erx_ec = []
for elt in erx_csv.loc[:, 'EC number']:
    for sub in elt.split(';'):
        erx_ec.append(sub)

swiss_ec = []
for elt in swissprot.loc[:, 'EC number']:
    for sub in elt.split(';'):
        swiss_ec.append(sub)

print(len(set(erx_ec) & set(swiss_ec)) / len(set(erx_ec) | set(swiss_ec)))
idx_intersect = list(set(erx_ec) & set(swiss_ec))

0.8378031154649108


In [12]:
# Overlap in EC numbers

first = 'swissprot'
second = 'erxprot'

first_df = pd.read_csv(f"../data/{first}/{first}.csv", delimiter='\t')
first_df.set_index('Entry', inplace=True)

second_df = pd.read_csv(f"../data/{second}/{second}.csv", delimiter='\t')
second_df.set_index('Entry', inplace=True)

first_ec = []
for elt in first_df.loc[:, 'EC number']:
    first_ec.append(elt.split(';')[0])

second_ec = []
for elt in second_df.loc[:, 'EC number']:
    second_ec.append(elt.split(';')[0])

print(len(set(first_ec) & set(second_ec)) / len(set(first_ec) | set(second_ec)))
print(len(set(first_ec) & set(second_ec)) / len(set(second_ec)))
ec_intersect = list(set(first_ec) & set(second_ec))

0.8055647020085611
0.8140076526368325


In [8]:
# Get consistent / discrepant ecs, update erxport
# w/ swissprot ec + extras
cons = []
discrep = []

for idx in idx_intersect:
    swiss = swissprot.loc[idx, 'EC number']
    erx = erx_csv.loc[idx, 'EC number']
    if sorted(swiss.split(';')) == sorted(erx.split(';')):
        updated = swiss.split(';')
        erx_csv.loc[idx, 'EC number'] = ';'.join(updated)
        cons.append(idx)
    else:
        discrep.append(idx)
        erx_extra = list(set(erx.split(';')) - set(swiss.split(';')))
        updated = swiss.split(';') + erx_extra
        erx_csv.loc[idx, 'EC number'] = ';'.join(updated)


erx_csv.to_csv('../data/erxprot/erxprot.csv', sep='\t')

In [9]:
len(cons), len(discrep), len(cons) / len(idx_intersect), len(discrep) / len(idx_intersect)

(41314, 481, 0.9884914463452567, 0.011508553654743391)

In [5]:
erx_csv.head()

,EC number,Sequence
Entry,,
P11766,1.1.1.1;1.1.1.284,MANEVIKCKAAVAWEAGKPLSIEEIEVAPPKAHEVRIKIIATAVCH...
P07327,1.1.1.1,MSTAGKVIKCKAAVLWELKKPFSIEEVEVAPPKAHEVRIKMVAVGI...
P28332,1.1.1.1,MSTTGQVIRCKAAILWKPGAPFSIEEVEVAPPKAKEVRIKVVATGL...
P00326,1.1.1.1,MSTAGKVIKCKAAVLWELKKPFSIEEVEVAPPKAHEVRIKMVAAGI...
P40394,1.1.1.105;1.1.1.1;1.1.1.66,MFAEIQIQDKDRMGTAGKVIKCKAAVLWEQKQPFSIEEIEVAPPKT...


In [6]:
swissprot.head()

,EC number,Sequence
Entry,,
P51451,2.7.10.2,MGLVSSKKPDKEKPIKEKDKGQWSPLKVSAQDKDAPPLPPLVVFNH...
C7C422,3.5.2.6,MELPNIMHPVAKLSTALAAALMLSGCMPGEIRPTIGQQMETGDQRF...
Q07457,2.3.2.27,MTAEPATKKIKLELSDPSEPLTQSDVIAFQKEALFRCINRRRVDFE...
Q07009,3.4.22.53,MAGIAMKLAKDREAAEGLGSHERAIKYLNQDYETLRNECLEAGALF...
Q9TTH8,3.4.22.54,MPTVISASVAPRTGAEPRSPGPIAQAAQGKGTEAGGGNPSGIYSAI...


In [14]:
# Check that current erxprot csv has same sequence as associate with erx's original files
erx_csv = pd.read_csv('../data/erxprot/erxprot.csv', delimiter='\t')
erx_csv.set_index('Entry', inplace=True)
ddir = '/home/stef/Downloads/proteins/'

ls = list(os.listdir(ddir))
# erx_original = {elt.split('|')[0].strip() : elt.split()}

for fn in os.listdir(ddir):
    foo = torch.load(ddir + fn)
    break
foo


{'label': 'A0A2D1BLU4 | 1.14.14.51',
 'mean_representations': {32: tensor([ 1.8712, -5.3498, -1.0984,  ...,  2.1770, -2.3396, -4.5177])}}